## Explicabilidade

Este trabalho tem como objetivo o treinamento de modelos explicáveis para o problema de análise de crimes segundo características socio-econômicas nas regiões de São Paulo. Aqui, retornamos ao uso de todas as variáveis socio-econômicas para obtenção de explicações completas acerca da origem e desenvolvimento criminal nas regiões. No entanto, utilizaremos uma arquitetura de modelos diferente das utilizadas nos trabalhos anteriores.

O desafio de prever crimes numa região poderia ser modelada como uma regressão para prever a distribuição dos boletins de ocorrência, por exemplo, no tempo. No entanto, regiões com baixo índice de criminalidade não possuem amostras de boletins de ocorrência capazes de se extrair uma distribuição que modele os BOs de forma satisfatória, ou até mesmo fiel. A forma que encontramos para minimizar este problema anteriormente foi de modelar o problema como um problema de classificação em regiões perigosas ou não, baseado apenas na quantidade de BOs registrados em dada região. No entanto, este tipo de redução carrega inúmeros vieses e não é justo para tomar qualquer tipo de decisão, principalmente devido a ambiguidade que um hard threshold causa ao diferenciar amostras cujo padrão é semelhante, mas o ground-truth induzido não é (por exemplo, classificar como perigoso regiões com mais de 1 BOs por dia em média classifica regiões com 0.99 BOs por dia como seguros.).

Pensando nisso e numa forma de se aprimorar o problema em si de forma que sua saída seja mais fácil de ser explicada, utilizamos um modelo híbrido de Regressão Inflada em zero. Esta abordagem melhora a capacidade de explicação ao separar regiões perigosas de não perigosas (zero e não-zero) e então obter um modelo de regressão para regiões perigosas, o que garante modelos capazes de quantificar a incidência criminal e não apenas sua existência.

### Revisão Bibliográfica

[Curiel et. al](https://link.springer.com/article/10.1007/s10940-017-9354-9) argumenta que a distribuição de ocorrências criminais ocorre segundo uma distribuição de Poisson-Binomial, onde cada indivíduo é selecionado por uma Binomial e então a ocorrência é modelada por uma distribuição de Poisson. Deste modo, escolhemos tratar nosso problema de forma que cada região é tratada como uma amostra de uma distribuição de Poisson que tenta prever a quantidade de BOs relatados naquela região. Uma regressão de Poisson utiliza um aproximador de esperança dado por

$$\mathbb{E}[y \,|\, x] = \exp(\theta^Tx)$$

Que induz uma distribuição de probabilidade condicional de Poisson

$$p_\theta(y \,|\, x) = \frac{\exp(y \cdot \theta^Tx)}{y!} \exp(-e^{\theta^Tx})$$

Os parâmetros $\theta$ são então otimizados de forma a maximizar a verossimilhança dos dados $\mathcal{D} = \{x^{(i)}, y^{(i)}\}_{i=1}^N$, 

$$\theta = \argmax_{\theta}\prod_{i=1}^N p_\theta(y_i \,|\, x_i)$$

Como trata-se de um problema de regressão em que 

In [149]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PoissonRegressor
from sklearn.pipeline import Pipeline, make_pipeline
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme('notebook', 'whitegrid', 'Set1')

In [160]:
data = pd.read_csv('../dataset/Padrão Urbano/PU.csv', sep=';')

X = data.drop(columns=['SETTT'])

categ_columns = ['HOMIC', 'ARISC', 'PMANC', 'EXURB', 'AGL91', 'AGL00', 'AGL10', 'DEN80', 'DEN91', 'DEN00', 'DEN10', 'CLUSTER']
log_columns = [feature for feature in X.columns if 'POP' in feature] + ['VIAGEM']

X['CLUSTER'] = OrdinalEncoder().fit_transform(X[['CLUSTER']])

preprocess = ColumnTransformer([
    ("log_numeric", make_pipeline(FunctionTransformer(np.log1p), StandardScaler()), log_columns),
    ("onehot_categ", OneHotEncoder(), categ_columns)
], remainder = StandardScaler())

X.head(5)

,VIAGEM,TDESL,HOMIC,ARISC,PMANC,EXURB,POP80,POP81,POP82,POP83,...,LIX10,PJM80,PJM91,PJM00,PJM10,VER80,VER91,VER00,VER10,CLUSTER
0,1428.9,29.8,1,0,0,2,1050,1043,1035,1027,...,0.9962,0.0810,0.0759,0.0658,0.0558,0.0407,0.0215,0.0260,0.0410,2.0
1,1554.2,21.7,1,0,0,1,1091,1079,1068,1057,...,1.0000,0.1006,0.0802,0.0818,0.0701,0.4207,0.0000,0.0068,0.0000,0.0
2,1471.5,25.8,1,0,0,2,963,960,956,952,...,1.0000,0.0935,0.0863,0.0840,0.0560,0.0433,0.0000,0.0531,0.0370,2.0
3,1404.8,27.7,2,0,0,2,1218,1183,1149,1115,...,1.0000,0.0904,0.0745,0.0658,0.0752,0.4054,0.1043,0.1602,0.1271,1.0
4,1492.0,28.6,2,0,0,2,1201,1181,1163,1146,...,1.0000,0.0954,0.0863,0.1101,0.0809,0.5530,0.0000,0.0575,0.0500,1.0


In [161]:
crimes = pd.read_csv('../dataset/Crimes/Listagem_Geral.csv', index_col=0)

crimes["DATA"] = pd.to_datetime(crimes["DATA"])

counts = crimes['SETOR'].value_counts()

y = data['SETTT'].apply(lambda setor : counts.get(setor, 0)).to_numpy()

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [163]:
frac = 0.6

n_aug_samples = int(len(X_train) * frac)

idx = np.random.randint(0, len(X_train), size=n_aug_samples)

X_aug = X_train.iloc[idx].copy()

X_aug["CAR10"] = X_train["CAR10"].sample(n=n_aug_samples, replace=True, random_state=21).to_numpy()

y_aug = y_train[idx]

X_aug = pd.concat([X_train, X_aug])
y_aug = np.hstack([y_train, y_aug])

### Baseline Models

In [145]:
def plot_labels(model, X, y, interval=(0, 100)):
    fig, ax = plt.subplots(figsize=(8, 5), dpi=200)

    n ,bins, patches = plt.hist(y, bins=np.linspace(interval[0], interval[1], 100), alpha=.5, label='Ground truth')
    plt.hist(model.predict(X), bins=bins, alpha=.5, label="Predicted")
    plt.xlim(*interval)
    plt.legend()
    plt.title("Distribution ")
    plt.xlabel('Number of reports')
    plt.ylabel('Count')
    plt.show()

In [192]:
from sklearn.model_selection import KFold

def evaluate(models, X, y, X_test, y_test, metrics, cv=20, df=None, figname=None):
    if df is None:
        df = pd.DataFrame(columns=metrics.keys())
    
    results = {}
    for i, (name, model) in enumerate(models.items()):
        scores = {metric : [] for metric in metrics}

        for train_idx, val_idx in KFold(n_splits=cv).split(X, y):
            model.fit(X.iloc[train_idx], y[train_idx])

            for metric_name, metric in metrics.items():
                scores[metric_name].append(metric(model, X.iloc[val_idx], y[val_idx]))

        results[name] = scores

        model.fit(X, y)

        result = {metric : scorer(model, X_test, y_test) for metric, scorer in metrics.items()}

        df.loc[name, :] = result

    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    ax.set_title('Algorithm Comparison')

    n_metrics = len(metrics)
    n_models  = len(models)

    names = list(models.keys())

    spacing = 0.1
    delta   = (1 - spacing)*(1 - 1/n_metrics)/2

    positions = np.hstack([np.linspace(i - delta, i + delta, n_metrics) for i in range(n_models)])
    boxes = plt.boxplot(np.array([model_result[metric] for model_result in results.values() for metric in metrics]).T, positions=positions, widths=1.5*delta/(n_metrics-1), patch_artist=True)

    ax.xaxis.set_major_formatter(lambda _, i : names[i//n_metrics] if i%n_metrics==1 else '')

    for i in range(n_metrics * n_models):
        color = sns.color_palette(n_colors=n_metrics)[i%n_metrics]

        boxes['boxes'][i].set_color((1, 1, 1, 0.4))
        boxes['boxes'][i].set(facecolor=color, alpha=0.8)
        boxes['medians'][i].set_color((0, 0, 0, 0.4))
        boxes['whiskers'][2*i].set_color(color)
        boxes['whiskers'][2*i+1].set_color(color)
        boxes['caps'][2*i].set_color(color)
        boxes['caps'][2*i+1].set_color(color)
        boxes['fliers'][i].set_color(color)
        boxes['fliers'][i].set_color(color)

    for i, metric in enumerate(metrics):
        boxes['boxes'][i].set_label(metric)

    plt.ylim((0, 1))
    plt.ylabel("Score")
    plt.legend(frameon=True, bbox_to_anchor=(1.1, 1.05))

    if figname is not None:
        plt.savefig(f"{figname}.png")

    plt.show()

    return df

def style(df, format_table):
    return df.style.highlight_max().format(format_table,)


#### Poisson Regression

In [199]:
from sklearn.metrics import get_scorer, pairwise_distances

def consistency_score(estimator, X, y, k_neigh=5, metric="euclidean"):
    distances = pairwise_distances(X, metric=metric)

    y_pred = estimator.predict(X)

    y_neigh = y_pred[np.argsort(distances, axis=1)[:, 1:k_neigh+1]].mean(axis=1)

    return 1 - np.abs(y - y_neigh).mean()

metrics = {
    "MAE" : get_scorer("neg_mean_absolute_error"),
    "MSE" : get_scorer("neg_mean_squared_error"),
    "Max Error" : get_scorer("max_error"),
    "Consistency" : consistency_score
}

format_table = {
    "MAE" : "{:}"
}

In [200]:
from sklearn.linear_model import PoissonRegressor, QuantileRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

poisson_model = Pipeline([
    ('preprocess', preprocess),
    ('model', PoissonRegressor(solver="newton-cholesky", alpha=1e-6))
])

quantile_model = QuantileRegressor(quantile=0.5, alpha=1e-5, solver='highs')

poisson_tree = HistGradientBoostingRegressor(loss="quantile", max_leaf_nodes=128, quantile=0.5)

models = {
    "Poisson Regressor"  : poisson_model,
    "Quantile Regressor" : quantile_model,
    "Poisson GBTree"     : poisson_tree
}

In [201]:
results = evaluate(models, X_aug, y_aug, X_test, y_test, metrics)

/home/danielgardin/.local/lib/python3.8/site-packages/sklearn/linear_model/_quantile.py:186: FutureWarning: The default solver will change from 'interior-point' to 'highs' in version 1.4. Set `solver='highs'` or to the desired solver to silence this warning.
  warnings.warn(


: 